In [1]:
# 勾配降下法をtensorFlowで実践してみる
# テキスト上は手動、自動微分での手法が載っているが、オプティマイザを使えばそのようなものは不要なためここではオプティマイザを最初から使う
# オプティマイザは最適化手法のこと
# 勾配降下法一つをとってもたくさんのオプティマイザが用意されているよう。　tensorflow.train.AdamOptimizer　が良さげ？

# 以下お決まり設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [4]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
# m : housing.data.shape の行が設定される　20640
# n : housing.data.shape の列が設定される  8

# housing.data : データの中身を見れる(多次元配列)
# hoousing.data.shape : 配列の定義を見れる ここでは(20640,8) 20640インスタンス、特徴量8個

In [5]:
# 勾配降下法はスケーリングが重要なのでまずはスケーリングを行う。手法はお任せ。ここではscikit-learnで実施する
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data) # housing.data を指定してスケーリングを行う
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
# np.ones((m, 1)) : [1.]の20640個の配列
# np.c_ : 行列の連結
# np.c_[np.ones((m, 1)), housing.data] : [1., housing.dataの特徴量(8個)] の20640インスタンスのnumpy配列が出来上がる
# 先頭にバイアスとして　１　が設定されるnumpy配列ができているということ

In [6]:
# axis = 0 : 列ごと
# axis = 1 : 行ごと

print(scaled_housing_data_plus_bias.mean(axis=0)) # 列ごとの平均値
# scaled_housing_data_plus_bias : [[1.,...]] の　(20640,9) のnumpy配列
# 列ごとの平均値は一つ目の特徴量は1固定なので1、以降の特徴量はその値による

print(scaled_housing_data_plus_bias.mean(axis=1)) # 行ごとの平均値
# こちらは行ごとの平均値であるため、インスタンスごとの平均値をもっている

print(scaled_housing_data_plus_bias.mean()) # 平均値
# インスタンスすべての平均値

print(scaled_housing_data_plus_bias.shape) # numpy配列の情報

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [27]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
# random_uniform : 与えた形状と範囲に基づいてランダム値を格納するテンソルを生成する
# tf.random_uniform([n + 1, 1]) では (9,1)のnumpy配列　ここでは操作名"theata" として(9,1)のnumpy配列 最小値-1,最大値1 で作成
# theataが重みをもった特徴量ベクトルになる

#y_pred = tf.matmul(X, theta, name="predictions")
# tf.matmul : 行列積  ここでは訓練データ(20640,9)と(9,1)の行列積となる python 3.5 からは以下の書き方がある
# ここで行列積を出しているのは入力となるnumpy配列の特徴量ベクトルと重み更新のためのtheata特徴量ベクトルをかけることにより予測値を出すため
#python3.5からの簡単な記載
y_pred = X @ theta

error = y_pred - y # 上で予測した値とラベルデータを比較して誤差をMSEに流し込む
mse = tf.reduce_mean(tf.square(error), name="mse") # 平均二乗誤差であるMSEの定義をしている

In [28]:
# オプティマイザとして勾配降下法のオプティマイザを設定
# 学習率をの設定をしていることに注意
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# 上で作成した誤差関数を訓練用に設定している
training_op = optimizer.minimize(mse)

In [29]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0: # 100 回終わるごとにメッセージを表示しているだけ
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval() # theataは重み更新の特徴量ベクトルなのでこの値が「重み」となっている

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.566705
Epoch 300 MSE = 0.5555719
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.5436362
Epoch 600 MSE = 0.5396294
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.5340678
Epoch 900 MSE = 0.5321474
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401658]
 [-0.34770882]
 [ 0.36178368]
 [ 0.00393811]
 [-0.04269556]
 [-0.6614528 ]
 [-0.6375277 ]]


<tf.Tensor 'X:0' shape=(20640, 9) dtype=float32>